In [12]:
import pandas as pd
import numpy as np
import uts
import time

Train Data

In [13]:
import os
# assign directory
directory = 'data/training'

frames = []

# iterate over files in the directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #print(f)
        df = pd.read_csv(f)
        df["filename"] = filename[:-4]       
        
        frames.append(df)

df = pd.concat(frames)

#reindex df
df.shape  #(26011, 10)
df.index = np.arange(len(df))

# df.loc[55]

        

Testing wordsegment

In [14]:
#df["FormElementName"].loc[20000:]

# to save time
# df = df.loc[20000:]
# df

# 20001         AnnualEstimate
# 20002     SolarFeedInDisplay
# 20003     TotalCostForPeriod
# 20004      RetailerContactNo
# 26007    AieosTeamsWebhookId > a i eos teams web hook id
# 26008             GetMessage 


In [15]:
# df = df.applymap(str)
# df = uts.splitWordSegment(df)

In [16]:
# df["FormElementName"].loc[20000:26008]
df["FormElementName"].loc[682:694]

682        ComparisonRate
683            Repayments
684             VedaScore
685    EstimatedRepayment
686        SelectAndApply
687              Progress
688               Payment
689             AccountId
690           Description
691             Scheduled
692             Processed
693             Principal
694              Interest
Name: FormElementName, dtype: object

In [17]:
df = df.applymap(str)
df = uts.splitWordSegment(df)

# # option 2: split
# for index, row in df.iterrows():
#     df.loc[index,'FormElementName'] = uts.splitAtUpperCase(row['FormElementName'])

gdf = df.groupby(['filename','ProjectName','FormName'])
# gdf = gdf[['filename', 'ProjectName', 'FormName', 'FormElementName', 'ProjectGuid', 'FormGuid', 'FormElementId']]
gdf = gdf[['filename', 'ProjectName', 'FormName', 'FormElementName']]


In [18]:
# checking forms with <= 1 form element
x = df.groupby(['filename','ProjectName','FormName'])['FormName'].count()
x = x[x<=1]
x

filename         ProjectName           FormName                     
AustralianUnity  CommunicationGateway  AieosChatReply                   1
                                       Support                          1
                                       UserPreference                   1
                 Finance               AccountDetailsParentComponent    1
                                       ApplicantParentComponent         1
                                                                       ..
VistaEnergy      Workflow              Scorecard                        1
                                       Stream                           1
                                       Workflow                         1
                                       WorkflowGroup                    1
                                       WorkflowNode                     1
Name: FormName, Length: 403, dtype: int64

In [19]:
# dfr=gdf.apply(lambda x: x)
# dfr.to_csv('result/grouped.csv')

In [20]:
gdf.get_group(('VistaEnergy','SmartDocHub', 'ClientMessageQueue'))

,filename,ProjectName,FormName,FormElementName
13807,VistaEnergy,SmartDocHub,ClientMessageQueue,message
13808,VistaEnergy,SmartDocHub,ClientMessageQueue,action
13809,VistaEnergy,SmartDocHub,ClientMessageQueue,path
13810,VistaEnergy,SmartDocHub,ClientMessageQueue,user profile id
13811,VistaEnergy,SmartDocHub,ClientMessageQueue,group


In [21]:
# gdf.get_group(('Nimble','SmartDocHub', 'ClientMessageQueue'))
# ('Saiver', 'SmartDocHub', 'ClientMessageQueue')
# ('Nimble', 'SmartDocHub', 'ClientMessageQueue')


Table (group) Head Encoding

In [22]:
# assert False

df to set

[Updated At] [  Party Id] [Created At] [Updated At] [    Job Id]
to  {'Updated', 'At', 'Party', 'Id', 'Created', 'At', 'Updated', 'At', 'Job', 'Id', 'Name'}

In [23]:
# df to one set
def df_set_new(gdf):

    drop_lst = []
    res = []    
    for table_name, df in gdf:
        
        df = df.dropna() 
                
        if len(df.index) <= 1:
            drop_lst.append(table_name)
            pass
        else:
            df_tmp = df[['FormElementName']]
        
            string =''
            for item in df_tmp['FormElementName']:
                string = string + ' ' + item
            
            lst = string.split()
            #print("lst: ",lst)
        
            res.append(set(lst))
            
        #print("todo: drop 0 or 1", len(df.index), table_name)  ## drop it and drop it from tn as well
        
    return res, drop_lst

lst, drop_lst = df_set_new(gdf)
lst[:2]


[{'chat',
  'create',
  'get',
  'hook',
  'identifier',
  'name',
  'new',
  'recipient',
  'teams',
  'user',
  'users',
  'web'},
 {'a',
  'eos',
  'get',
  'hook',
  'i',
  'id',
  'message',
  'resource',
  'responded',
  'subscription',
  'teams',
  'user',
  'web'}]

In [24]:
lst[:2]

[{'chat',
  'create',
  'get',
  'hook',
  'identifier',
  'name',
  'new',
  'recipient',
  'teams',
  'user',
  'users',
  'web'},
 {'a',
  'eos',
  'get',
  'hook',
  'i',
  'id',
  'message',
  'resource',
  'responded',
  'subscription',
  'teams',
  'user',
  'web'}]

In [25]:
#### Table dictionary
tn = pd.DataFrame(gdf).loc[:,0]

# tn == series
tn_lst = []

for idx, item in enumerate(tn):
    if item in drop_lst:
        pass
        #print(idx,item)
    else:    
        nlst = [idx]
        #tuple to list
        nlst = nlst + list(item) 
        nlst.append(item)
        tn_lst.append(nlst) 
        
tn_lst

tn = []
for row in tn_lst:
    tn.append(row[0])
tn  

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 13,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 23,
 24,
 25,
 27,
 28,
 30,
 32,
 34,
 35,
 36,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 87,
 88,
 89,
 90,
 91,
 92,
 94,
 95,
 96,
 98,
 99,
 101,
 102,
 103,
 104,
 108,
 109,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 136,
 138,
 140,
 141,
 143,
 144,
 146,
 147,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 169,
 170,
 171,
 172,
 173,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 210,
 211,
 212,
 214,


Calculate Intersection Matrix

In [26]:
def create_inetrsection_matrix(lst_set):
    mtr = []

    for fe1 in lst_set:
        row = []
        for fe2 in lst_set:            
            u = fe1.union(fe2) ### error
            i = fe1.intersection(fe2)
            
            
            # if len(u) == 0:
            #     print(fe1, fe2)
            
            
            dist = 0
            if not u: # empty set
                print(u)
            else:
                dist = 1- (len(i) / len(u)) #todo division by zero  
                            
            
            row.append(dist)
        mtr.append(row)

    matrix = pd.DataFrame.from_records(mtr)
    return matrix

In [ ]:
s = time.time()

# create matrix
matrix = create_inetrsection_matrix(lst)
matrix.set_axis(tn, axis=1, inplace=True)
matrix.set_axis(tn, axis=0, inplace=True)
matrix

assert False


/tmp/ipykernel_2637261/1877500400.py:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  matrix.set_axis(tn, axis=1, inplace=True)
/tmp/ipykernel_2637261/1877500400.py:6: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  matrix.set_axis(tn, axis=0, inplace=True)


,1,2,3,4,5,6,7,8,10,11,...,3432,3433,3434,3435,3436,3437,3438,3439,3440,3441
1,0.000000,0.750000,0.857143,0.944444,0.923077,0.920000,0.944444,0.944444,1.0,1.000000,...,0.920000,0.973684,0.961538,0.965517,0.944444,0.952381,0.955556,0.957447,0.956522,0.958333
2,0.750000,0.000000,0.730769,0.823529,0.925000,0.880000,1.000000,1.000000,1.0,0.869565,...,0.923077,1.000000,1.000000,0.966667,0.947368,0.954545,0.956522,0.958333,0.913043,0.916667
3,0.857143,0.730769,0.000000,0.961538,0.913043,0.906250,1.000000,1.000000,1.0,0.777778,...,0.970588,0.931818,0.939394,0.944444,0.920000,0.965517,0.962264,0.944444,0.967742,0.968750
4,0.944444,0.823529,0.961538,0.000000,0.942857,0.900000,0.923077,1.000000,1.0,1.000000,...,0.900000,1.000000,1.000000,0.958333,0.923077,0.866667,0.923077,0.926829,0.812500,0.823529
5,0.923077,0.925000,0.913043,0.942857,0.000000,0.714286,0.972222,0.942857,1.0,0.951220,...,0.928571,0.903846,0.953488,0.933333,0.972222,0.947368,0.934426,0.919355,0.950000,0.951220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3437,0.952381,0.954545,0.965517,0.866667,0.947368,0.863636,0.300000,0.866667,1.0,1.000000,...,0.809524,0.972222,0.958333,0.962963,0.937500,0.000000,0.953488,0.955556,1.000000,1.000000
3438,0.955556,0.956522,0.962264,0.923077,0.934426,0.936170,0.975610,0.975610,1.0,1.000000,...,0.936170,0.912281,0.888889,0.960784,0.975610,0.953488,0.000000,0.243902,0.978261,0.978723
3439,0.957447,0.958333,0.944444,0.926829,0.919355,0.938776,0.976744,0.976744,1.0,0.979592,...,0.938776,0.896552,0.893617,0.962264,0.976744,0.955556,0.243902,0.000000,0.957447,0.958333
3440,0.956522,0.913043,0.967742,0.812500,0.950000,0.961538,1.000000,1.000000,1.0,0.958333,...,0.961538,0.973684,1.000000,1.000000,1.000000,1.000000,0.978261,0.957447,0.000000,0.076923


In [ ]:
matrix.head()

# matrix to csv
matrix.to_csv('result/res_matrix_3_new_header.csv')

e = time.time()
print("took: ", e-s)


took:  23.475823640823364


In [ ]:
tndf = pd.DataFrame(tn_lst, columns=['Id','Compose', 'Project', 'Form', 'Concat'])
tndf.to_csv("result/table_name_list.csv", index=False)
pd.DataFrame(drop_lst).to_csv("result/dropped_form_list.csv", index=False)


Matrix to tree

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio.Phylo.TreeConstruction import _Matrix
from Bio.Phylo.TreeConstruction import DistanceMatrix
from Bio import Phylo

# create a for loop 1 to 4 
    

In [ ]:
ddf = pd.read_csv('result/res_matrix_3_new_header.csv',keep_default_na=False)

ddf = ddf.iloc[:,1:]

# making triangular shape
names = ddf.columns.tolist()
lst = ddf.values.tolist()
for idx, row in enumerate(lst):
    del row[idx+1:]
    
    
assert False

AssertionError: 

In [ ]:
s = time.time()

df = lst
dm = DistanceMatrix(names=names, matrix=df)
constructor = DistanceTreeConstructor()
upgmatree = constructor.upgma(dm)

#upgmatree.ladderize()  # Flip branches so deeper clades are displayed at top
#Phylo.draw_ascii(upgmatree)

e = time.time()
print("took: ", e-s)


took:  7356.690273284912


In [ ]:
from Bio.Phylo import NewickIO

s = time.time()
print(s)

with open('result/nwtree_1403.txt', 'w') as file:
    fileNo = NewickIO.write([upgmatree],file)
    print(fileNo)

tree_text = ""
with open('result/nwtree_1403.txt', 'r') as file:
    tree_text = file.read().replace('\n', '')
    
    
e = time.time()
print("took: ", e-s)


1678768962.7392504
1
took:  0.04871082305908203


In [ ]:
import json

s = time.time()

dict = {
    "newick": tree_text
}
json_obj = json.dumps(dict, indent=4)

with open("d3treeview-app/src/tree.json","w") as outfile:
    outfile.write(json_obj)
    
e = time.time()
print("took: ", e-s)

took:  0.0012774467468261719


In [ ]:
from pycirclize import Circos
from io import StringIO
from Bio import Phylo

s = time.time()


# Load newick tree
tree = Phylo.read(StringIO(tree_text), "newick")

# Initialize circos sector by tree size
circos = Circos(sectors={"Tree": tree.count_terminals()})
sector = circos.sectors[0]

# Plot tree
track = sector.add_track((50, 100))
track.axis(ec="lightgrey")
track.tree(tree, leaf_label_size=12)

fig = circos.plotfig()

e = time.time()
print("took: ", e-s)

ModuleNotFoundError: No module named 'pycirclize'